# AWS Rekognition Object/Scene Detection API
Nora Webb Williams | August 15, 2018

### Purpose:

To demonstrate how to run images through the AWS Rekognition API to do Object/Scene Detection. The resulting csv has a row for each label assigned to each image with greater than 50% confidence.

### Modules

In [10]:
import csv
import boto3 
import pickle
import os

### Paths

In [11]:
########### Paths
### MUST ADJUST HERE (1/2)
# Path to where your want to save the resulting labels
rekog_results_dir = 'path_to_where_you_want_to_save_labels'
# e.g.:
#rekog_results_dir = 'C:/Users/Nora/Desktop/auto_tagger_example/results/'

# Path to where your images are
rekog_images_dir = 'path_to_where_your_images_are'
# e.g.:
#rekog_images_dir = 'C:/Users/Nora/Desktop/auto_tagger_example/data/'

### Connect to Rekognition API

In [12]:
# Read in your personal keys
# You can hard code your access key ID and secret key ID into the script, 
# but this is not recommended

personal_access_key = "your_personal_access_key"
secret_access_key = "your_secret_access_key"

# Instead we recommend storing your keys securely in a csv or text file.
# For example, if you have saved your keys in a csv:

credentials = []

### MUST ADJUST HERE (2/2)
with open('path_to_your_saved_AWS_access_keys.csv', newline='') as csvfile:

# e.g.:
#with open('C:/Users/Nora/Desktop/auto_tagger_example/keys/AWS_personal_nora_admin_credentials.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        credentials.append(row)

personal_access_key = credentials[0]['Access key ID']
secret_access_key = credentials[0]['Secret access key']

# Initialize the boto client to access the Rekogniton api
client=boto3.client('rekognition','us-east-1', # or choose the best region for your work, 
                                               # e.g. the location of your S3 bucket if using that method to store images
                    aws_access_key_id = personal_access_key, 
                    aws_secret_access_key = secret_access_key) 

### Create a list of images to run through the API

In [13]:
# Make a list of all the images in the rekog_data_dir you created
local_images = os.listdir(rekog_images_dir)

### Run each image through the API and store the results

In [14]:
##### Detect labels
## 
holder_labels = []

for imageFile in local_images:
    with open(rekog_images_dir + imageFile, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
    
    print('Detected labels for ' + imageFile)
    
    ## If no labels detected, still save the info:
    if len(response['Labels']) == 0:
        print ("No Labels Detected")
        temp_dict = {}
        temp_dict["image_id"] = imageFile
        temp_dict["full_detect_labels_response"] = response
        temp_dict["label_num"] = ''
        temp_dict["label_str"] = ''
        temp_dict["label_conf"] = ''
        temp_dict["label_orient_correct"] = response['OrientationCorrection']
        holder_labels.append(temp_dict)   
    
    else:
        
        label_counter = 1
        
        for label in response['Labels']:
            print (label['Name'] + ' : ' + str(label['Confidence']))
            temp_dict = {}
            temp_dict["image_id"] = imageFile
            temp_dict["full_detect_labels_response"] = response
            temp_dict["label_num"] = label_counter
            temp_dict["label_str"] = label['Name']
            temp_dict["label_conf"] = label['Confidence']
            temp_dict["label_orient_correct"] = response['OrientationCorrection']
            label_counter +=1 # update for the next label
            holder_labels.append(temp_dict)
          
len(holder_labels)


Detected labels for ex_1.jpg
Coffee Cup : 98.23609924316406
Cup : 98.23609924316406
Road : 60.62405014038086
Freeway : 52.599571228027344
Highway : 52.599571228027344
Building : 50.51565933227539
City : 50.51565933227539
Downtown : 50.51565933227539
Town : 50.51565933227539
Urban : 50.51565933227539
Detected labels for ex_2.jpg
Human : 99.13925170898438
People : 99.13923645019531
Person : 99.13925170898438
Bar Counter : 98.43019104003906
Pub : 98.43019104003906
Diner : 97.12300109863281
Food : 97.12300109863281
Meal : 97.12300109863281
Restaurant : 97.12300109863281
Worker : 83.39073181152344
Animal : 76.40896606445312
Aquarium : 76.40896606445312
Sea Life : 76.40896606445312
Water : 76.40896606445312
Lighting : 51.80131149291992
Cafe : 50.84360885620117
Apparel : 50.51062774658203
Clothing : 50.51062774658203
Detected labels for ex_3.jpg
Brochure : 88.77820587158203
Flyer : 88.77820587158203
Paper : 88.77820587158203
Poster : 88.77820587158203
Collage : 56.89072036743164
Logo : 54.578

55

### Write out the results to a csv

In [15]:
# Write out the results to a csv
with open(rekog_results_dir + 'awsrekognition_detect_labels.csv', 'w', newline = '') as csvfile:
    fieldnames = ['image_id', 'full_detect_labels_response',
                  'label_num', 'label_str',
                  'label_conf', 'label_orient_correct'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader() 
    for entry in holder_labels:
        writer.writerow(entry)